In [8]:
from pathlib import Path
repo_path = Path('/home/ricardo/ABUS2023_documents/tdsc_abus23')

In [13]:
import numpy as np
import SimpleITK as sitk
import pandas as pd
from torchvision.transforms import (
    Compose,
    Resize,
    InterpolationMode,
)
from PIL import Image
from tqdm import tqdm

In [14]:
metadata_path = repo_path / 'data/challange_2023/Test/metadata.csv'
metadata = pd.read_csv(metadata_path)

In [16]:
for case_id in metadata['case_id']:
    print(case_id)

100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
